In [2]:
import numpy as np
import cv2
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None

In [3]:
import numpy as np
import cv2
import os
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)

        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False

In [4]:
import xml.etree.ElementTree as ET

def read_anntation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

In [5]:
from os import listdir
import cv2
import numpy as np

def read_train_dataset(dir):
    images = []
    annotations = []

    for file in listdir(dir):
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_anntation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)

    return images, annotations

In [8]:
import imgaug as ia
from imgaug import augmenters as iaa
from files import *
from pascal_voc_writer import Writer
import re

ia.seed(1)

dir ='/Users/ksjljk1030/drug_image3/sample/' 
dir2 = '/Users/ksjljk1030/drug_image3/annotation3/'
dir3 = '/Users/ksjljk1030/drug_image3/real_image3/'
images, annotations = read_train_dataset(dir)
for idx in range(len(images)):
    for k in range(14):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))
        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)


        seq = iaa.Sequential([
            #iaa.Multiply((1.2, 1.5)),
            iaa.Affine(
                translate_px={"x": 40, "y": 60},
                scale=(0.4, 1.0),
                rotate=(-90, 90)
            ),
            #iaa.GaussianBlur(sigma=(0, 3.0)),
            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)
            #iaa.SimplexNoiseAlpha(iaa.OneOf([
            #            iaa.EdgeDetect(alpha=(0.5, 1.0)),
            #            iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
            #        ])
            #)
        ])

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
        
        #print(annotations[idx][1].rstrip('.xml'))
        #print(annotations[idx][2].rstrip('.jpg'))
        
        #new_image_file = dir3 + annotations[idx][2].rsplit('. ')[0]+'_'+str(k+1)+'.jpg'
        new_image_file = dir3 + re.sub('.jpg','',  annotations[idx][2])+'_'+str(k+1)+'.jpg'
        imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        voc_writer.save(dir2 + annotations[idx][1].rstrip('.xml')+'_'+str(k+1)+'.xml')
        print(str(idx)+" "+str(k))
        
        """
        for i in range(len(bbs.bounding_boxes)):
            before = bbs.bounding_boxes[i]
            after = bbs_aug.bounding_boxes[i]
            print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
                i,
                before.x1, before.y1, before.x2, before.y2,
                after.x1, after.y1, after.x2, after.y2)
            )
        """

        #image_before = bbs.draw_on_image(image, thickness=20)
        #image_after = bbs_aug.draw_on_image(image_aug, thickness=20)

        #cv2.imshow('image_before', image_before)
        #cv2.imshow('image_after', image_after)

        cv2.waitKey(0)

C:\Users\ksjljk1030\AppData\Local\Temp\ipykernel_22112\578532253.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images = np.array(images)


0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
11 13
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
12 12
12 13
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
13 13
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
15 0
15 1
15 2
15 3
15 4
15 5


110 9
110 10
110 11
110 12
110 13
111 0
111 1
111 2
111 3
111 4
111 5
111 6
111 7
111 8
111 9
111 10
111 11
111 12
111 13
112 0
112 1
112 2
112 3
112 4
112 5
112 6
112 7
112 8
112 9
112 10
112 11
112 12
112 13
113 0
113 1
113 2
113 3
113 4
113 5
113 6
113 7
113 8
113 9
113 10
113 11
113 12
113 13
114 0
114 1
114 2
114 3
114 4
114 5
114 6
114 7
114 8
114 9
114 10
114 11
114 12
114 13
115 0
115 1
115 2
115 3
115 4
115 5
115 6
115 7
115 8
115 9
115 10
115 11
115 12
115 13
116 0
116 1
116 2
116 3
116 4
116 5
116 6
116 7
116 8
116 9
116 10
116 11
116 12
116 13
117 0
117 1
117 2
117 3
117 4
117 5
117 6
117 7
117 8
117 9
117 10
117 11
117 12
117 13
118 0
118 1
118 2
118 3
118 4
118 5
118 6
118 7
118 8
118 9
118 10
118 11
118 12
118 13
119 0
119 1
119 2
119 3
119 4
119 5
119 6
119 7
119 8
119 9
119 10
119 11
119 12
119 13
120 0
120 1
120 2
120 3
120 4
120 5
120 6
120 7
120 8
120 9
120 10
120 11
120 12
120 13
121 0
121 1
121 2
121 3
121 4
121 5
121 6
121 7
121 8
121 9
121 10
121 11
121 12
121 1

203 12
203 13
204 0
204 1
204 2
204 3
204 4
204 5
204 6
204 7
204 8
204 9
204 10
204 11
204 12
204 13
205 0
205 1
205 2
205 3
205 4
205 5
205 6
205 7
205 8
205 9
205 10
205 11
205 12
205 13
206 0
206 1
206 2
206 3
206 4
206 5
206 6
206 7
206 8
206 9
206 10
206 11
206 12
206 13
207 0
207 1
207 2
207 3
207 4
207 5
207 6
207 7
207 8
207 9
207 10
207 11
207 12
207 13
208 0
208 1
208 2
208 3
208 4
208 5
208 6
208 7
208 8
208 9
208 10
208 11
208 12
208 13
209 0
209 1
209 2
209 3
209 4
209 5
209 6
209 7
209 8
209 9
209 10
209 11
209 12
209 13
210 0
210 1
210 2
210 3
210 4
210 5
210 6
210 7
210 8
210 9
210 10
210 11
210 12
210 13
211 0
211 1
211 2
211 3
211 4
211 5
211 6
211 7
211 8
211 9
211 10
211 11
211 12
211 13
212 0
212 1
212 2
212 3
212 4
212 5
212 6
212 7
212 8
212 9
212 10
212 11
212 12
212 13
213 0
213 1
213 2
213 3
213 4
213 5
213 6
213 7
213 8
213 9
213 10
213 11
213 12
213 13
214 0
214 1
214 2
214 3
214 4
214 5
214 6
214 7
214 8
214 9
214 10
214 11
214 12
214 13
215 0
215 1
215 2


297 3
297 4
297 5
297 6
297 7
297 8
297 9
297 10
297 11
297 12
297 13
298 0
298 1
298 2
298 3
298 4
298 5
298 6
298 7
298 8
298 9
298 10
298 11
298 12
298 13
299 0
299 1
299 2
299 3
299 4
299 5
299 6
299 7
299 8
299 9
299 10
299 11
299 12
299 13
300 0
300 1
300 2
300 3
300 4
300 5
300 6
300 7
300 8
300 9
300 10
300 11
300 12
300 13
301 0
301 1
301 2
301 3
301 4
301 5
301 6
301 7
301 8
301 9
301 10
301 11
301 12
301 13
302 0
302 1
302 2
302 3
302 4
302 5
302 6
302 7
302 8
302 9
302 10
302 11
302 12
302 13
303 0
303 1
303 2
303 3
303 4
303 5
303 6
303 7
303 8
303 9
303 10
303 11
303 12
303 13
304 0
304 1
304 2
304 3
304 4
304 5
304 6
304 7
304 8
304 9
304 10
304 11
304 12
304 13
305 0
305 1
305 2
305 3
305 4
305 5
305 6
305 7
305 8
305 9
305 10
305 11
305 12
305 13
306 0
306 1
306 2
306 3
306 4
306 5
306 6
306 7
306 8
306 9
306 10
306 11
306 12
306 13
307 0
307 1
307 2
307 3
307 4
307 5
307 6
307 7
307 8
307 9
307 10
307 11
307 12
307 13
308 0
308 1
308 2
308 3
308 4
308 5
308 6
308 7
30

...

2645 13
2646 0
2646 1
2646 2
2646 3
2646 4
2646 5
2646 6
2646 7
2646 8
2646 9
2646 10
2646 11
2646 12
2646 13
2647 0
2647 1
2647 2
2647 3
2647 4
2647 5
2647 6
2647 7
2647 8
2647 9
2647 10
2647 11
2647 12
2647 13
2648 0
2648 1
2648 2
2648 3
2648 4
2648 5
2648 6
2648 7
2648 8
2648 9
2648 10
2648 11
2648 12
2648 13
2649 0
2649 1
2649 2
2649 3
2649 4
2649 5
2649 6
2649 7
2649 8
2649 9
2649 10
2649 11
2649 12
2649 13
2650 0
2650 1
2650 2
2650 3
2650 4
2650 5
2650 6
2650 7
2650 8
2650 9
2650 10
2650 11
2650 12
2650 13
2651 0
2651 1
2651 2
2651 3
2651 4
2651 5
2651 6
2651 7
2651 8
2651 9
2651 10
2651 11
2651 12
2651 13
2652 0
2652 1
2652 2
2652 3
2652 4
2652 5
2652 6
2652 7
2652 8
2652 9
2652 10
2652 11
2652 12
2652 13
2653 0
2653 1
2653 2
2653 3
2653 4
2653 5
2653 6
2653 7
2653 8
2653 9
2653 10
2653 11
2653 12
2653 13
2654 0
2654 1
2654 2
2654 3
2654 4
2654 5
2654 6
2654 7
2654 8
2654 9
2654 10
2654 11
2654 12
2654 13
2655 0
2655 1
2655 2
2655 3
2655 4
2655 5
2655 6
2655 7
2655 8
2655 9
2655

2726 4
2726 5
2726 6
2726 7
2726 8
2726 9
2726 10
2726 11
2726 12
2726 13
2727 0
2727 1
2727 2
2727 3
2727 4
2727 5
2727 6
2727 7
2727 8
2727 9
2727 10
2727 11
2727 12
2727 13
2728 0
2728 1
2728 2
2728 3
2728 4
2728 5
2728 6
2728 7
2728 8
2728 9
2728 10
2728 11
2728 12
2728 13
2729 0
2729 1
2729 2
2729 3
2729 4
2729 5
2729 6
2729 7
2729 8
2729 9
2729 10
2729 11
2729 12
2729 13
2730 0
2730 1
2730 2
2730 3
2730 4
2730 5
2730 6
2730 7
2730 8
2730 9
2730 10
2730 11
2730 12
2730 13
2731 0
2731 1
2731 2
2731 3
2731 4
2731 5
2731 6
2731 7
2731 8
2731 9
2731 10
2731 11
2731 12
2731 13
2732 0
2732 1
2732 2
2732 3
2732 4
2732 5
2732 6
2732 7
2732 8
2732 9
2732 10
2732 11
2732 12
2732 13
2733 0
2733 1
2733 2
2733 3
2733 4
2733 5
2733 6
2733 7
2733 8
2733 9
2733 10
2733 11
2733 12
2733 13
2734 0
2734 1
2734 2
2734 3
2734 4
2734 5
2734 6
2734 7
2734 8
2734 9
2734 10
2734 11
2734 12
2734 13
2735 0
2735 1
2735 2
2735 3
2735 4
2735 5
2735 6
2735 7
2735 8
2735 9
2735 10
2735 11
2735 12
2735 13
2736 0
2

2806 11
2806 12
2806 13
2807 0
2807 1
2807 2
2807 3
2807 4
2807 5
2807 6
2807 7
2807 8
2807 9
2807 10
2807 11
2807 12
2807 13
2808 0
2808 1
2808 2
2808 3
2808 4
2808 5
2808 6
2808 7
2808 8
2808 9
2808 10
2808 11
2808 12
2808 13
2809 0
2809 1
2809 2
2809 3
2809 4
2809 5
2809 6
2809 7
2809 8
2809 9
2809 10
2809 11
2809 12
2809 13
2810 0
2810 1
2810 2
2810 3
2810 4
2810 5
2810 6
2810 7
2810 8
2810 9
2810 10
2810 11
2810 12
2810 13
2811 0
2811 1
2811 2
2811 3
2811 4
2811 5
2811 6
2811 7
2811 8
2811 9
2811 10
2811 11
2811 12
2811 13
2812 0
2812 1
2812 2
2812 3
2812 4
2812 5
2812 6
2812 7
2812 8
2812 9
2812 10
2812 11
2812 12
2812 13
2813 0
2813 1
2813 2
2813 3
2813 4
2813 5
2813 6
2813 7
2813 8
2813 9
2813 10
2813 11
2813 12
2813 13
2814 0
2814 1
2814 2
2814 3
2814 4
2814 5
2814 6
2814 7
2814 8
2814 9
2814 10
2814 11
2814 12
2814 13
2815 0
2815 1
2815 2
2815 3
2815 4
2815 5
2815 6
2815 7
2815 8
2815 9
2815 10
2815 11
2815 12
2815 13
2816 0
2816 1
2816 2
2816 3
2816 4
2816 5
2816 6
2816 7
28

2887 5
2887 6
2887 7
2887 8
2887 9
2887 10
2887 11
2887 12
2887 13
2888 0
2888 1
2888 2
2888 3
2888 4
2888 5
2888 6
2888 7
2888 8
2888 9
2888 10
2888 11
2888 12
2888 13
2889 0
2889 1
2889 2
2889 3
2889 4
2889 5
2889 6
2889 7
2889 8
2889 9
2889 10
2889 11
2889 12
2889 13
2890 0
2890 1
2890 2
2890 3
2890 4
2890 5
2890 6
2890 7
2890 8
2890 9
2890 10
2890 11
2890 12
2890 13
2891 0
2891 1
2891 2
2891 3
2891 4
2891 5
2891 6
2891 7
2891 8
2891 9
2891 10
2891 11
2891 12
2891 13
2892 0
2892 1
2892 2
2892 3
2892 4
2892 5
2892 6
2892 7
2892 8
2892 9
2892 10
2892 11
2892 12
2892 13
2893 0
2893 1
2893 2
2893 3
2893 4
2893 5
2893 6
2893 7
2893 8
2893 9
2893 10
2893 11
2893 12
2893 13
2894 0
2894 1
2894 2
2894 3
2894 4
2894 5
2894 6
2894 7
2894 8
2894 9
2894 10
2894 11
2894 12
2894 13
2895 0
2895 1
2895 2
2895 3
2895 4
2895 5
2895 6
2895 7
2895 8
2895 9
2895 10
2895 11
2895 12
2895 13
2896 0
2896 1
2896 2
2896 3
2896 4
2896 5
2896 6
2896 7
2896 8
2896 9
2896 10
2896 11
2896 12
2896 13
2897 0
2897 1
2

2967 10
2967 11
2967 12
2967 13
2968 0
2968 1
2968 2
2968 3
2968 4
2968 5
2968 6
2968 7
2968 8
2968 9
2968 10
2968 11
2968 12
2968 13
2969 0
2969 1
2969 2
2969 3
2969 4
2969 5
2969 6
2969 7
2969 8
2969 9
2969 10
2969 11
2969 12
2969 13
2970 0
2970 1
2970 2
2970 3
2970 4
2970 5
2970 6
2970 7
2970 8
2970 9
2970 10
2970 11
2970 12
2970 13
2971 0
2971 1
2971 2
2971 3
2971 4
2971 5
2971 6
2971 7
2971 8
2971 9
2971 10
2971 11
2971 12
2971 13
2972 0
2972 1
2972 2
2972 3
2972 4
2972 5
2972 6
2972 7
2972 8
2972 9
2972 10
2972 11
2972 12
2972 13
2973 0
2973 1
2973 2
2973 3
2973 4
2973 5
2973 6
2973 7
2973 8
2973 9
2973 10
2973 11
2973 12
2973 13
2974 0
2974 1
2974 2
2974 3
2974 4
2974 5
2974 6
2974 7
2974 8
2974 9
2974 10
2974 11
2974 12
2974 13
2975 0
2975 1
2975 2
2975 3
2975 4
2975 5
2975 6
2975 7
2975 8
2975 9
2975 10
2975 11
2975 12
2975 13
2976 0
2976 1
2976 2
2976 3
2976 4
2976 5
2976 6
2976 7
2976 8
2976 9
2976 10
2976 11
2976 12
2976 13
2977 0
2977 1
2977 2
2977 3
2977 4
2977 5
2977 6
2